# Importações

In [31]:
from IPython.display import clear_output
%pip install docplex
clear_output()

In [32]:
import docplex.mp.model as mp
from docplex.mp.model_reader import ModelReader
import pandas as pd

# Utils

In [33]:
def get_vars(model:mp.Model):
  x_vars = model.find_matching_vars('x_')
  o_vars = model.find_matching_vars('o_')
  return (x_vars, o_vars)

In [34]:
def insert_output_constraints(
  model:mp.Model, output_variables, network_output, binary_variables
):
  mdl = model
  # binary_variables = mdl.binary_var_list(keys = [f'z_{j}' for j in range(len(output_variables)-1)], lb=0, ub=1)
  variable_output = output_variables[network_output]
  aux_var = 0
  indicators_constraints = []
  for i, output in enumerate(output_variables):
    if i != network_output:
      p = binary_variables[aux_var]
      aux_var += 1
      indicators_constraints.append(mdl.add_indicator(p, variable_output <= output, 1))
  return indicators_constraints

In [35]:
def check_explanation_truth(
  model:mp.Model, 
  objective,
  output_network:int,
  explanation: dict[str, float],
  n_classes: int):
  # O := o_i > o_j p/ todo j com i!=j
  # not O :=  o_i <= o_j p/ algum j com i!=j
  # adicionar restricoes de implicação
  
  input_vars, output_vars = get_vars(model)
  
  # C
  input_constraints = model.add_constraints([
    model.get_var_by_name(x) == explanation[x] 
    for i, x in enumerate(explanation)],names="input")
  
  # O
  binary_variables_outputs = model.binary_var_list(n_classes - 1, name="b") #type: ignore  
  model.add_constraint(model.sum(binary_variables_outputs) >= 1) #type: ignore
  output_constraints = insert_output_constraints(
    model, 
    output_vars, 
    output_network, 
    binary_variables_outputs)
  
  if objective is not None:
    model.maximize(objective)
  
  solution = model.solve(log_output=False)
  
  if objective is not None:
    model.remove_objective()
  
  
  # model.remove_constraints(output_constraints)
  # model.remove_constraints(input_constraints)
    
  # C and F |= O  
  # sse
  # C and F and (not O) is insat.
  if solution is not None:  #   se o modelo é satisfazível.
    return False, solution  #   "instancia" não é uma solução correta
  else:
    return True, None       #   "instancia" é uma solução correta

# Configurações

In [36]:
n_layers = 3
model_file_h5 = f'model_{n_layers}layers_20neurons.h5'
dataset_name = 'digits' 
n_classes = 10
file = f'datasets/{dataset_name}/results/models/{model_file_h5}/relaxed.lp'
results_file = f'datasets/{dataset_name}/results/models/{model_file_h5}/df.csv'
data_file = f'datasets/{dataset_name}/data.csv'

data_df = pd.read_csv(data_file)
results_df = pd.read_csv(results_file)

model = ModelReader.read(file, ignore_names=False)

In [41]:
x, o = get_vars(model)

for x_i, o_i in zip(x, o):
  print(x_i.ub)

In [52]:
model.add_constraint(x[0]==0.5)

docplex.mp.LinearConstraint[](x_0,EQ,0.5)

In [58]:
network_input = [[2 for i in range(64)]] 

In [59]:
# x_vars = mdl.find_matching_vars('x_')
x_values = [feature for i, feature in enumerate(network_input[0])]

In [60]:
x_values

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [51]:
x[0].equals(x[0])

True

In [7]:
instancia = data_df.iloc[0].tolist()
target = instancia[-1]
features_values:list[float] = instancia[:-1]
explanation = {f'x_{i}': x for i, x in enumerate(features_values)}
# remover features que nao estao na explicaca
# laço entre todas as instancias do dataframe de resultados, tratando os dados

In [8]:
response = check_explanation_truth(
  model = model.copy(),  
  objective=None,
  explanation = explanation,
  output_network = int(target), 
  n_classes = n_classes)

if response[0] is False:
  print('-*'*10, "Solução errada", '-*'*10)
  response[1].display()
else:
  print("Solução correta")

* No objective to optimize - searching for a feasible solution
Solução correta


In [9]:

from graphviz import Digraph

g = Digraph('G', filename='cluster.gv')

# NOTE: the subgraph name needs to begin with 'cluster' (all lowercase)
#       so that Graphviz recognizes it as a special cluster subgraph

with g.subgraph(name='cluster_0') as c:
    c.attr(style='filled', color='lightgrey')
    c.node_attr.update(style='filled', color='white')
    c.edges([('a0', 'a1'), ('a1', 'a2'), ('a2', 'a3')])
    c.attr(label='process #1')

with g.subgraph(name='cluster_1') as c:
    c.attr(color='blue')
    c.node_attr['style'] = 'filled'
    c.edges([('b0', 'b1'), ('b1', 'b2'), ('b2', 'b3')])
    c.attr(label='process #2')

g.edge('start', 'a0')
g.edge('start', 'b0')
g.edge('a1', 'b3')
g.edge('b2', 'a3')
g.edge('a3', 'a0')
g.edge('a3', 'end')
g.edge('b3', 'end')

g.node('start', shape='Mdiamond')
g.node('end', shape='Msquare')

g.view()


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH